In [ ]:



# 如何实现对某网站内容作为知识库，实现问答机器人




In [7]:
from langchain_community.document_loaders import RecursiveUrlLoader

loader = RecursiveUrlLoader("https://python.langchain.com/docs/integrations/document_loaders/", max_depth=1)
documents = loader.load()


In [8]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(documents)


In [9]:
# pip install -qU langchain-google-genai

In [10]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain_google_genai import GoogleGenerativeAIEmbeddings

embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore(embeddings)

In [12]:
document_ids = vector_store.add_documents(documents=all_splits)

In [14]:
retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)
retriever.invoke("Document Loader")

[Document(id='0e358bf9-b9c4-4b8b-be02-f04206f3af33', metadata={'source': 'https://python.langchain.com/docs/integrations/document_loaders/', 'content_type': 'text/html; charset=utf-8', 'title': 'Document loaders | 🦜️🔗 LangChain', 'description': 'DocumentLoaders load data into the standard LangChain Document format.', 'language': 'en'}, page_content='<!-- -->\n<p>DocumentLoaders load data into the standard LangChain Document format.</p>\n<p>Each DocumentLoader has its own specific parameters, but they can all be invoked in the same way with the .load method.\nAn example use case is as follows:</p>')]

In [21]:
import getpass
import os

if not os.environ.get("GOOGLE_API_KEY"):
  os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter API key for Google Gemini: ")

from langchain.chat_models import init_chat_model

model = init_chat_model("gemini-2.0-flash", model_provider="google_genai")

In [ ]:
# 参考文档：
# https://python.langchain.com/api_reference/langchain/chains/langchain.chains.retrieval.create_retrieval_chain.html


In [23]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain import hub


retrieval_qa_chat_prompt = hub.pull("langchain-ai/retrieval-qa-chat")


combine_docs_chain = create_stuff_documents_chain(
    model, retrieval_qa_chat_prompt
)

retrieval_chain = create_retrieval_chain(retriever, combine_docs_chain)

retrieval_chain.invoke({"input": "..."})

{'input': '...',
 'context': [Document(id='53c96d92-f701-4a12-b4ae-f10af675e79f', metadata={'source': 'https://python.langchain.com/docs/integrations/document_loaders/', 'content_type': 'text/html; charset=utf-8', 'title': 'Document loaders | 🦜️🔗 LangChain', 'description': 'DocumentLoaders load data into the standard LangChain Document format.', 'language': 'en'}, page_content='</body>\n</html>')],
 'answer': 'I am sorry, but I cannot answer any questions based on the context you provided because the context is empty. It only contains the basic HTML structure `<html> <body> </body> </html>` without any actual content.'}

In [25]:
query = "关于Document Loader你能检索到哪些知识？"
response = retrieval_chain.invoke({"input": query})
print(response["answer"])


根据提供的上下文，我能检索到以下关于Document Loader的知识：

*   存在一个链接到 `/docs/integrations/document_loaders/pandas_dataframe/`，表明有关于使用 Pandas DataFrame 作为 Document Loader 的信息。
*   存在一个 `/docs/integrations/document_loaders/parsers/azure_openai_whisper_parser/` 链接，暗示有关于 Document Loader 的解析器（parsers）的信息，特别是与 Azure OpenAI Whisper 相关。
*   存在一个链接到 `/docs/integrations/document_loaders/pdfminer/`，表明有关于使用 PDFMinerLoader 作为 Document Loader 的信息。


In [28]:
query = "关于Document Loader你能检索到哪些知识？"
response = model.invoke(query)
print(response)


content='好的，关于 Document Loader，我能检索到的知识点主要包括以下几个方面：\n\n**1. 什么是 Document Loader？**\n\n*   **定义：** Document Loader 是一种工具/库/模块，用于从各种来源加载文档数据，并将其转换为统一的格式，以便后续处理，例如文本分析、信息检索、机器学习等。\n*   **作用：**\n    *   **数据提取：** 从不同类型的文档中提取文本内容。\n    *   **数据转换：** 将提取的文本转换为统一的格式，例如字符串或文档对象。\n    *   **数据预处理：**  有些 Document Loader 可能会进行一些基本的预处理，例如去除空格、换行符等。\n*   **重要性：** Document Loader 是许多 NLP (自然语言处理) 和数据分析流程中的关键第一步，因为它可以将原始文档数据转换为可用的格式。\n\n**2. Document Loader 的类型和支持的文档格式：**\n\nDocument Loader 可以支持多种文档格式，常见的包括：\n\n*   **文本文件：**\n    *   `.txt` (纯文本文件)\n    *   `.csv` (逗号分隔值文件)\n    *   `.md` (Markdown 文件)\n*   **办公文档：**\n    *   `.pdf` (PDF 文件)\n    *   `.docx` (Microsoft Word 文件)\n    *   `.xlsx` (Microsoft Excel 文件)\n    *   `.pptx` (Microsoft PowerPoint 文件)\n    *   `.odt` (OpenDocument Text)\n    *   `.ods` (OpenDocument Spreadsheet)\n    *   `.odp` (OpenDocument Presentation)\n*   **网页：**\n    *   `.html` (HTML 文件)\n*   **代码文件：**\n    *   `.py` (Python 文件)\n    *   `.java` (Java 文件)\n    *   `.js` 